In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append['..']
import myd2l

In [3]:
class Residual(nn.Module):
    def __init__(self, in_channels, n_channels, use_1x1conv=False, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, n_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(n_channels, n_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels, n_channels, kernel_size=1, stride=stride) if use_1x1conv else None
        self.bn1 = nn.BatchNorm2d(n_channels)
        self.bn2 = nn.BatchNorm2d(n_channels)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        X = self.conv3(X) if self.conv3 else X
        Y += X

        return F.relu(Y)


In [5]:
blk = Residual(3, 3)
X = torch.rand(4, 3, 5, 5)
Y = blk(X)
print(Y.shape)

blk = Residual(3, 6, use_1x1conv=True, stride=2)
Y = blk(X)
print(Y.shape)

torch.Size([4, 3, 5, 5])
torch.Size([4, 6, 3, 3])


In [6]:
def resnet_block(in_channels, n_channels, n_residuals, first_block=False):
    blk = []
    for i in range(n_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, n_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(n_channels, n_channels))
    
    return blk

In [7]:
b1 = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

net = nn.Sequential(
    b1, b2, b3, b4, b5,
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(), nn.Linear(512, 10)
)

In [8]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, X.shape)

Sequential torch.Size([1, 64, 56, 56])
Sequential torch.Size([1, 64, 56, 56])
Sequential torch.Size([1, 128, 28, 28])
Sequential torch.Size([1, 256, 14, 14])
Sequential torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d torch.Size([1, 512, 1, 1])
Flatten torch.Size([1, 512])
Linear torch.Size([1, 10])


In [ ]:
batch_size, resize = 256, 224
train_iter, test_iter = myd2l.load_data_fashion_mnist(batch_size, resize)

n_epochs, lr = 10, 0.05
device = torch.device('cuda')
myd2l.train_clf(net, n_epochs, lr, train_iter, test_iter, device)